# Proyecto: Machine Learning
---
*Fecha de Realización: Mar-2023*

- **Senior Data Scientist:** Francisco Alfaro
- **Instructor:** Alfonso Tobar
- **Code Reviewer:** Iván Hernández
- **Notebook by:** Julio C. Martínez

# Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.
Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del curso de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.
Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

# Instrucciones del proyecto.
---

Para este proyecto estaremos trabajando con los siguientes puntos:

1. Abrir y examinar el archivo de datos.
2. Segmentar los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
3. Investigar la calidad de diferentes modelos cambiando los hiperparámetros. Describir brevemente los hallazgos del estudio.
4. Comprobar la calidad del modelo usando el conjunto de prueba.
5. Tarea adicional: hacer una prueba de cordura al modelo. Estos datos son más complejos que los que habíamos usado antes así que no será una tarea fácil.

## Carga de Datos y Estudio General de la Información.
---

Cargar liberías.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
import matplotlib.pyplot as plt

Cargar archivos.

In [ ]:
clientes = pd.read_csv('/datasets/users_behavior.csv')

Cantidad de características y objetivo.

In [ ]:
clientes.shape

(3214, 5)

Tipo de características y objetivo.

In [ ]:
clientes.dtypes

calls       float64
minutes     float64
messages    float64
mb_used     float64
is_ultra      int64
dtype: object

Visualización del dataset.

In [ ]:
clientes.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Información General.

In [ ]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**OBSERVACIONES**

**Descripción de los Datos**

Cada observación en el dataset contiene información del comportamiento mensual sobre el usuario. La información dada es la siguiente:

- `сalls` — número de llamadas,
- `minutes` — duración total de la llamada en minutos,
- `messages` — número de mensajes de texto,
- `mb_used` — Tráfico de Internet utilizado en MB,
- `is_ultra` — plan para el mes actual (Ultra - 1, Smart - 0).

> Nuestros datos están en orden. En la siguiente sección vamos a traer nuestro modelo entrenado.

## Segmentación de Datos.
---

En este apartado vamos a dividir nuestra matriz en tres partes para obtener lo siguiente:

- Set de entrenamiento (60%)
- Set de validación (20%)
- Set de prueba (20%)

Se realizará de esta manera dado que no existe un dataset de prueba y debemos crearlo previo al entrenamiento del dataset fuente. Para hacer nuestra segmentación primero vamos a definir nuestras características y nuestro objetivo de la siguiente manera:

In [ ]:
features = clientes.drop(['is_ultra'], axis=1) # Conjunto de características.
target   = clientes['is_ultra']                # Objetivo Selección de Modelo.

Realizamos split para obtener conjunto de entrenamiento (80%) y prueba (20%).

In [ ]:
X_rest, X_test, y_rest, y_test = train_test_split(
    features,
    target,
    test_size = 0.20,
    random_state=12345)

Verificamos split.

In [ ]:
X_rest.shape, y_rest.shape, X_test.shape, y_test.shape

((2571, 4), (2571,), (643, 4), (643,))

Ahora obtenemos conjunto de validación (20%) usando el 80% del conjunto resultante anterior para obtener el 60%.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_rest,
    y_rest,
    test_size=0.25,
    random_state=12345)

Comprobamos split.

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((1928, 4), (1928,), (643, 4), (643,))

Enlistamos nuestros sets de prueba, entrenamiento y validación.

In [ ]:
print(
    'Dataset de Entrenamiento (60%):', X_train.shape, y_train.shape,'\n',
    'Dataset de Prueba (20%):'       , X_test.shape , y_test.shape,'\n',
    'Dataset de Validación (20%):'   , X_val.shape  , y_val.shape
)

Dataset de Entrenamiento (60%): (1928, 4) (1928,) 
 Dataset de Prueba (20%): (643, 4) (643,) 
 Dataset de Validación (20%): (643, 4) (643,)


Comprobación del Split.

In [ ]:
print('Dataset Fuente:'     , clientes.shape[0]),
print('Total de los Splits:', X_train.shape[0] + X_test.shape[0] + X_val.shape[0])

Dataset Fuente: 3214
Total de los Splits: 3214


## Investigar la Calidad de los Diferentes Modelos.
---

Ya que tenemos nuestros datasets listos, ahora vamos a escoger el mejor modelo a entrenar para esta tarea. En primer lugar debemos considerar que nuestra tarea es de *Clasificación* por lo cual que debemos enfocarnos en ello.

Para las tareas de clasificación tenemos 3 modelos que son:

- árbol de decisión.
- regresión logística.
- bosque aleatorio.

Para nuestro caso vamos a escoger el modelo de regresión logística porque su velocidad de ejecuciòn es más rápida y  porque tiene una precisión intermedia que por lo regular no tiene sobreajusto y nos puede funcionar. Además este modelo contribuye para la aplicación de eventos binarios, es decir, eventos donde ocurre una u otra opción así que vamos a nuestro modelo.

### Modelo Regresión Logística.

In [ ]:
# Seleccionamos modelo de regresión logística.
rl_model = LogisticRegression(random_state=12345)
# Entrenamos nuestro modelo.
rl_model.fit(X_train, y_train)

LogisticRegression(random_state=12345)

Mostrar la Exactitud del Modelo.

In [ ]:
rl_score_test = rl_model.score(X_test, y_test)

print('Exactitud del modelo de regresión logística en el conjunto de prueba:', rl_score_test)

Exactitud del modelo de regresión logística en el conjunto de prueba: 0.7589424572317263


**Observaciones**

> Los datos nos muestran que nuestro modelo tiene una gran exactitud justo como lo habíamos predicho aunque el modelo de entrenamiento es poco más preciso que el modelo de validación. Posiblemente sea el mejor modelo que podamos escoger pero vamos a intentar con otro modelo para estar seguros.

### Bosque Aleatorio.

El modelo que selecionamos ahora será un bosque aletario debido a que consta de un gran nivel de exactitud, el problema es que la velocidad de ejecución es lenta, cuantos más árboles haya más lento será este modelo.

Encontremos el Mejor Modelo.

In [ ]:
best_score = 0
best_est   = 0

for est in range(1,31):
    ba_model = RandomForestClassifier(random_state=12345, n_estimators=est) # Configurar No de árboles.
    ba_model.fit(X_train, y_train)                                          # Entrenamos Modelo
    ba_model_score = ba_model.score(X_test, y_test)
    if ba_model_score > best_score:
        best_score += 1   # Mejor puntuación de exactitud.
        best_est   += 1   # Mejores estimaciones correspondientes a la mejor puntuación de exactitud.

print('Exactitud del mejor modelo en el conjunto de validación (n_estimators = {}):{}'.format(best_est, best_score))

Exactitud del mejor modelo en el conjunto de validación (n_estimators = 1):1


In [ ]:
print('El mejor modelo del conjunto con n_estimartors = 1 es:', ba_model_score)

El mejor modelo del conjunto con n_estimartors = 1 es: 0.7853810264385692


**Observaciones**

> La exactitud de nuestro modelo es excelente pero nuestro umbral de exactitud para este proyecto es de 0.75 así que esta fuera de los límites. Por lo tanto vamos nos quedaremos con el modelo de regresión líneal.

### Árbol de Decisión.

Dentro de sus características se encuentran el tener una baja precisión aunque cuenta con la velocidad más alta de ejecución. Vamos a investigarlo y veamos qué resultados nos puede proporcionar.

In [ ]:
ad_model = DecisionTreeClassifier(random_state=12345) # Seleccionamos Modelo.
ad_model.fit(X_train, y_train)                        # Entrenamos Modelo.

DecisionTreeClassifier(random_state=12345)

Comprobamos Calidad.

In [ ]:
ad_model.score(X_test, y_test)

0.7309486780715396

Encontremos el mejor modelo modificando la profundidad.

In [ ]:
best_model = None
best_result = 0

for depth in range(1, 5):
    ad_model = DecisionTreeClassifier(random_state=12345, max_depth = depth) # creamos modelo con la profundidad proporcionada
    ad_model.fit(X_train, y_train) # entrena el modelo
    predictions = ad_model.predict(X_test) # obtén las predicciones del modelo
    result = accuracy_score(y_test, predictions) # calcula la exactitud
    if result > best_result:
        best_model = ad_model
        best_result = result

print("Exactitud del mejor modelo en el conjunto de validacion:", best_result)

Exactitud del mejor modelo en el conjunto de validacion: 0.7869362363919129


**Observaciones**

> En un listado que realizamos ajustando la profundidad nuestro modelo nos indica que el mejor modelo de validación tiene un 78% de exactitud.

## Comprobar la Calidad

---

En el punto anterior averiguamos cuál de todos los modelos cumplía con tener el umbral de 0.75, ahora esta sección vamos a poner a prueba nuestro modelo de regresión logística para conocer su funcionamiento con un dataset que nuestro modelo no conoce, en este caso el de validación.

Definimos prediciones.

In [ ]:
## Recordemos que el nombre de nuestro modelo es rl_model.
X_predict = rl_model.predict(X_test)

Verificamos la Calidad del Modelo

In [ ]:
LR_result = accuracy_score(y_test, X_predict)
print('Exactitud del modelo en el conjunto de validación:', LR_result)

Exactitud del modelo en el conjunto de validación: 0.7589424572317263


## Prueba de Cordura.

---

Vamos a realizar nuestra prueba de cordura seleccionando el mejor modelo que encontramos anteriormente, recordemos que tenemos un umbral de 0.75 de exactitud. Se entiende que por umbral no debemos rebasar este límite de exactitud, es decir que no debemos ir más allá  del límite de 0.75 por lo tanto nuestro modelo a elegir debe ser el de Regresión Logística.

In [ ]:
# Entrenar Nuevo Modelo.
lo_regression = LogisticRegression(random_state=12345)

# Entrenamos con X_rest
# Esta variable contiene los conjuntos de X_train y X_test, es decir el 80% de nuestros datos.
lo_regression.fit(X_rest,y_rest)

# Obtenemos las predicciones en el conjunto de validación.
y_pred = lo_regression.predict(X_val)

# Obtenemos Score de Exactitud y RECM.
acc  = accuracy_score(y_val, y_pred)
recm = mean_squared_error(y_val, y_pred, squared=False) # Métrica extra.
print('La exactitud final de nuestro modelo es:', acc)
print('RECM del modelo de regresión lineal en el conjunto de validación:', recm)

La exactitud final de nuestro modelo es: 0.7262830482115086
RECM del modelo de regresión lineal en el conjunto de validación: 0.5231796553656224


**CONCLUSIONES**

> Nuestra prueba de cordura nos muestra una exactitud final de 0.72 pts respecto al modelo entrenado que fue de 0.75 es decir hay una variación de 3 pts en nuestro modelo.